In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 15.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 w

In [ ]:
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model
import torch

train_df = pd.read_csv("filtered_train.csv")
valid_df = pd.read_csv("filtered_valid.csv")
train_dataset = Dataset.from_pandas(train_df)
eval_dataset = Dataset.from_pandas(valid_df)

def format_prompt(examples):
    return {
        "text": [f"Transliterate this: {word}\nOutput: {output}" for word, output in zip(examples['Input'], examples['Output'])],
        "labels": examples['Output']
    }

train_dataset = train_dataset.map(format_prompt, batched=True, remove_columns=["Input", "Output"])
eval_dataset = eval_dataset.map(format_prompt, batched=True, remove_columns=["Input", "Output"])

print("Formatted training example:", train_dataset[0])


Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Formatted training example: {'lang_code': 'hin', '__index_level_0__': 0, 'text': 'Transliterate this: janamdivas\nOutput: जन्मदिवस', 'labels': 'जन्मदिवस'}


In [ ]:
print("Formatted training example:", eval_dataset[0])

Formatted training example: {'lang_code': 'hin', '__index_level_0__': 0, 'text': 'Transliterate this: spike\nOutput: स्पाइक', 'labels': 'स्पाइक'}


In [ ]:
model_name = "sarvamai/sarvam-1"
tokenizer = AutoTokenizer.from_pretrained(model_name)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
def tokenize_function(examples):
    inputs = tokenizer(examples["text"], padding="max_length", truncation=True, max_length=128)
    labels = tokenizer(examples["labels"], padding="max_length", truncation=True, max_length=128)
    inputs["labels"] = labels["input_ids"]
    return inputs

train_dataset = train_dataset.map(tokenize_function, batched=True, remove_columns=["text", "labels"])
eval_dataset = eval_dataset.map(tokenize_function, batched=True, remove_columns=["text", "labels"])

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/775k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/1.94M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/8.51M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto"
)

config.json:   0%|          | 0.00/717 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/21.0k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.77G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/279M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/193 [00:00<?, ?B/s]

In [ ]:
for name, param in model.named_parameters():
    print(name)

model.embed_tokens.weight
model.layers.0.self_attn.q_proj.weight
model.layers.0.self_attn.k_proj.weight
model.layers.0.self_attn.v_proj.weight
model.layers.0.self_attn.o_proj.weight
model.layers.0.mlp.gate_proj.weight
model.layers.0.mlp.up_proj.weight
model.layers.0.mlp.down_proj.weight
model.layers.0.input_layernorm.weight
model.layers.0.post_attention_layernorm.weight
model.layers.1.self_attn.q_proj.weight
model.layers.1.self_attn.k_proj.weight
model.layers.1.self_attn.v_proj.weight
model.layers.1.self_attn.o_proj.weight
model.layers.1.mlp.gate_proj.weight
model.layers.1.mlp.up_proj.weight
model.layers.1.mlp.down_proj.weight
model.layers.1.input_layernorm.weight
model.layers.1.post_attention_layernorm.weight
model.layers.2.self_attn.q_proj.weight
model.layers.2.self_attn.k_proj.weight
model.layers.2.self_attn.v_proj.weight
model.layers.2.self_attn.o_proj.weight
model.layers.2.mlp.gate_proj.weight
model.layers.2.mlp.up_proj.weight
model.layers.2.mlp.down_proj.weight
model.layers.2.inp

In [ ]:
from peft import LoraConfig, get_peft_model
import torch

config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.1,
    target_modules=["q_proj", "v_proj"]
)
model = get_peft_model(model, config)
model.print_trainable_parameters()

trainable params: 3,211,264 || all params: 2,528,299,008 || trainable%: 0.1270


In [ ]:
training_args = TrainingArguments(
    output_dir="./sarvam-transliteration",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_dir="./logs",
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=8,
    num_train_epochs=5,
    learning_rate=3e-5,
    fp16=True,
    save_total_limit=2,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: abhavana0410 (abhavana0410-ssn-coe) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Epoch,Training Loss,Validation Loss
1,1.899300,No log
2,0.055600,No log
3,0.050100,No log
4,0.046500,No log
5,0.045300,No log


TrainOutput(global_step=3125, training_loss=0.34492975006103516, metrics={'train_runtime': 5571.8984, 'train_samples_per_second': 8.974, 'train_steps_per_second': 0.561, 'total_flos': 9.173139456e+16, 'train_loss': 0.34492975006103516, 'epoch': 5.0})

In [ ]:
model.save_pretrained("./sarvam-transliteration")
tokenizer.save_pretrained("./sarvam-transliteration")

('./sarvam-transliteration/tokenizer_config.json',
 './sarvam-transliteration/special_tokens_map.json',
 './sarvam-transliteration/tokenizer.model',
 './sarvam-transliteration/added_tokens.json',
 './sarvam-transliteration/tokenizer.json')

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

# Path to your fine-tuned model
fine_tuned_model_path = "./sarvam-transliteration"

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(fine_tuned_model_path)
model = AutoModelForCausalLM.from_pretrained(fine_tuned_model_path)

# Ensure padding token is set if needed
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
def transliterate(word):
    input_text = f"Transliterate in Hindi script : {word}   \nOutput:"
    inputs = tokenizer(input_text, return_tensors="pt")
    output = model.generate(**inputs, max_length=30)
    transliteration = tokenizer.decode(output[0], skip_special_tokens=True).split("Output:")[-1].strip()
    print(transliteration)

    return transliteration


In [ ]:
test_words = ["dollars","school","computer","milligram"]
for word in test_words:
    print(f"Input: {word} → Transliteration: {transliterate(word)}")


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


डॉलर
Input: dollars → Transliteration: डॉलर


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


स्कूल
Input: school → Transliteration: स्कूल


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


कंप्यूटर
Input: computer → Transliteration: कंप्यूटर
मिलीग्राम
Input: milligram → Transliteration: मिलीग्राम


In [ ]:
transliteration_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer)

input_text = "Transliterate word to Tamil: Computer\nOutput:"
output = transliteration_pipeline(input_text, max_length=30)
transliteration = output[0]["generated_text"].split("Output:")[-1].strip()

print(f"Transliteration: {transliteration}")


Device set to use cuda:0


Transliteration: கம்ப்யூட்டர்,
